This note works an example of partial pooling from equation 12.1 of Gelman, Hill, "Data Analysis Using Regression and Multilevel/Hierarchical Models", Cambridge 2007.

## Introduction

The topic of formula 12.1 is: how to partially pool data by hand, though this task is often handled by a hierarchical model solver. 

The idea is: we want to estimate the ideal mean of a value from a few observations at a location. The optimal linear un-biased estimator is to just take the average of the observed values, and hope this is close to the ideal unobserved mean.

The wrinkle in partial pooling is: if we have data from other (different buy related) locations, can we use that to improve our estimate?

Let's work this example with a few simplifying assumptions, using the `Python` `sympy` package to do the algebra.

In [1]:
# import packages
import sympy
from sympy.stats import E, Normal
import numpy as np
import pandas as pd

Our idea is:

  * Each location has an unobserved mean value of examples drawn from this location, call this `LocationValue_j`.
  * The locations are related, in that the `LocationValue_j`s are all drawn from some common distribution. This is why we think pooling data could be useful.
  * For a given location, we figure some combination of the observations from the location, plus observations from other locations may be a lower expected error estimate than can be found using only observations from the location.

To execute this idea we need to define a great number of variables and their relations as follows.

In [2]:
# define initial variables
MeanLocationValue = sympy.Symbol("MeanLocationValue")  # center of distribution generating garages, unobserved
n_obs = sympy.Symbol("n_j", positive=True)  # total number of observations across all garages, observed.
BetweenLocationsSD = sympy.Symbol("BetweenLocationsSD")  # how garages very from each other in expected behavior, unobserved
ObservedMean = Normal("ObservedMean", mean=MeanLocationValue, std = BetweenLocationsSD / sympy.sqrt(n_obs))  # mean of all observations, observed
LocationValue_j = sympy.Symbol("LocationValue_j")  # actual expected behavior of a given garage, unobserved and the goal to estimate
LocationDistFactor_j = Normal("LocationDistFactor_j", mean=0, std=BetweenLocationsSD)  # how locations differ from each other, unobserved
def_LocationValue_j = MeanLocationValue + LocationDistFactor_j  #  generative definition of LocationDistFactor_j
PerObservationSD = sympy.Symbol("PerObservationSD", positive=True)  # sd of distribution generating observations, unobserved
n_j = sympy.Symbol("n_j", positive=True)  # number of observations at the j-th garage, observed
LocationMean_j = sympy.Symbol("LocationMean_j", mean=LocationValue_j, std=PerObservationSD)  # mean of all observations at garage j, observed
LocationCenterNoise_ji = Normal("LocationCenterNoise_ji", mean=0, std = PerObservationSD / sympy.sqrt(n_j))  # how mean observations at given location vary, unobserved
def_LocationMean_j = LocationValue_j + LocationCenterNoise_ji  # generative definition of LocationCenterNoise_ji
w = sympy.Symbol("w", positive=True)  # our weighting term picking how to pool specific and general observations, to solve for
estimate_j = sympy.Symbol("estimate_j")  # our estimate of the behavior of the j-th garage, to solve for
def_estimate_j = w * LocationMean_j + (1-w) * ObservedMean  # definition of our estimate
expected_error_term = LocationValue_j - estimate_j  # error of our estimate, to minimize square of

Formula 12.1 from Gelman and Hill is as follows.

<img src="IMG_1323.png">

This is a big equation with a "pooling strength" coefficient expanded into the expression. If we write the pooling coefficient as `w`, we can then re-write the above as the follosing.

In [3]:
def_estimate_j

LocationMean_j*w + (1 - w)*ObservedMean

That is: our estimate is `w` times using the observed per-location mean (`LocationMean_j`, the obvious estimate) plus `(1-w)` times the observed mean of all observations from all locations. Setting `w = 1` gives us the traditional "use only observations from the chosen location" solution. Setting `w = 0` gives us complete pooling, assuming there is not difference between locations. The trick is to find a `w` between `0` and `1` that might have lower expected square error. 

The 12.1 solution is a solution that picks `w` as the following.

In [4]:
neat_soln_approx = 1 / (1 + PerObservationSD**2 / (n_j * BetweenLocationsSD**2))

neat_soln_approx

1/(1 + PerObservationSD**2/(BetweenLocationsSD**2*n_j))

This solution for `w` has some nice properties.

  * `w` goes to `1` (the standard simple solution) as `PerObservationSD` goes to zero. This can be read as: "there is no point in pooling of their is already little uncertainty in the obvious estimate.
  * `w` goes to `1` (the standard simple solution) as `n_j` goes to infinity. This can be read as: "there is no point in pooling if we already have a lot of data for the obvious estimate.
  * `w` goes to `0` (combining all the data) as `PerObservationSD` goes to infinity. This can be read as: "combine all the data if the per-location uncertainty is very high."

The goal is to then derive this solution. First we will derive a similar solution, and then the identical solution.


## The exact solution

We can solve for `w` by performing some substitutions in to our error term. The goal is to minimize the square of this error term, which counts negative errors as also being bad.

The error term can be written as `A - B - C` where `A`, `B`, and `C` are as follows.

In [5]:
A = (1-w) * LocationDistFactor_j
B = w * LocationCenterNoise_ji
C = (1 - w) * (ObservedMean - MeanLocationValue)

We can confirm this as follows

In [6]:
error_term_exact = (
    expected_error_term
        .subs(estimate_j, def_estimate_j)  # definition of estimate_j
        .subs(LocationMean_j, def_LocationMean_j)  # generative definition of LocationMean_j
        .subs(LocationValue_j, def_LocationValue_j)  # generative definition of LocationValue_j
).expand().simplify()
assert (error_term_exact - (A - B - C)).expand() == 0
assert (E(C*C) - (1 - w)**2 * BetweenLocationsSD**2 / n_j).expand() == 0
assert E(A).expand() == 0
assert E(B).expand() == 0
assert E(C).expand() == 0

We are assuming each of the random variables `A`, `B`, and `C` are independent of the others and expected value `0`. This means `E[(A - B - C)**2] = E[A**2] + E[B**2] + E[C**2]`. We can then solve for where the derivative of this is zero to get the optimal value for `w`.

In [7]:
soln_exact = sympy.solve(sympy.diff((E(A * A) + E(B * B) + E(C * C)).expand(), w), w)[0]

soln_exact

BetweenLocationsSD**2*(n_j + 1)/(BetweenLocationsSD**2*n_j + BetweenLocationsSD**2 + PerObservationSD**2)

We can neaten this solution for `w` up a bit as follows.

In [8]:
neat_soln_exact = 1 / (1 + PerObservationSD**2 / ((n_j + 1) * BetweenLocationsSD**2))
assert (soln_exact - neat_soln_exact).together().expand() == 0

neat_soln_exact

1/(1 + PerObservationSD**2/(BetweenLocationsSD**2*(n_j + 1)))

Some algebra shows that this differs from the Gelman and Hill solution only in that we have an `(n_j + 1)` where they have an `n_j`.

## Reproducing the Gelman and Hill solution

We can match the Gelman and Hill solution by, during the solving, replacing the visible `MeanLocationValue` with our estimated `ObservedMean` (ignoring the small difference between them).

When we solve in that matter we get the Gelman and Hill `w` as follows.

In [9]:
error_term_approx = (
    expected_error_term
        .subs(estimate_j, def_estimate_j)  # definition of estimate_j
        .subs(ObservedMean, MeanLocationValue)  # this step is an approximation, using the unobserved MeanLocationValue as if it is the observed ObservedMean
        .subs(LocationMean_j, def_LocationMean_j)  # generative definition of LocationMean_j
        .subs(LocationValue_j, def_LocationValue_j)   # generative definition of LocationValue_j
).expand().simplify()


We claim this error term is equal to `A - B`, for the following `A`, `B`.

In [10]:
A = (1-w) * LocationDistFactor_j
B = w * LocationCenterNoise_ji


In [11]:
assert (error_term_approx - (A - B)).simplify() == 0
assert E(A).expand() == 0
assert E(B).expand() == 0

Again, we can expand `E[(A - B)**2]` as `E[A**2] + E[B**2]` (using the independence and mean-zero properties).

In [12]:
soln_approx = sympy.solve(sympy.diff(E(A**2) + E(B**2), w), w)[0]

soln_approx

BetweenLocationsSD**2*n_j/(BetweenLocationsSD**2*n_j + PerObservationSD**2)

In [13]:
assert (soln_approx - neat_soln_approx).together().expand() == 0

neat_soln_approx

1/(1 + PerObservationSD**2/(BetweenLocationsSD**2*n_j))

And this, as promised matches the text book. I would suggest a slight preference for the exact solution over this one, thought the differences are small.

## Working an Example

Let's see the partial pooling inference in action. 

We will generate the data according to the above hierarchical design. We will then see how a pooled estimate, itself using estimate parameters, can in fact improve results.

In [14]:
rng = np.random.default_rng(2023)  # set state of pseudo random number generator

In [15]:
# set parameters to specific values
example_between_locations_sd = 3.0
example_per_observations_sd = 10.0
n_locations = 10
n_obs_per_location = 2

In [16]:
# generate the unobserved location centers
example_location_value_mean = rng.normal(loc=0, scale=15, size=1)
example_locations = pd.DataFrame({
    "location_id": range(n_locations),
    "effect": rng.normal(loc=example_location_value_mean, scale=example_between_locations_sd, size=n_locations)
})

In [17]:
example_locations

,location_id,effect
0,0,12.480676
1,1,4.947432
2,2,9.691985
3,3,6.698217
4,4,11.451937
5,5,8.429935
6,6,4.289738
7,7,7.137952
8,8,7.862549
9,9,9.176378


In [18]:
# generate the data
observations = []
for j in range(example_locations.shape[0]):
    obs_j = pd.DataFrame({
        "location_id": j,
        "observation_id": range(n_obs_per_location),
        "observations": rng.normal(loc=example_locations.effect[j], scale=example_per_observations_sd, size=n_obs_per_location),
    })
    observations.append(obs_j)
observations = pd.concat(observations, ignore_index=True)


In [19]:
observations

,location_id,observation_id,observations
0,0,0,3.410240
1,0,1,13.802057
2,1,0,18.996457
3,1,1,8.988453
4,2,0,-0.682149
5,2,1,2.273959
6,3,0,19.333211
7,3,1,-0.195023
8,4,0,18.532085
9,4,1,11.101462


In [20]:
# define the standard estimator
def standard_effect_estimate_1_held_out(*, location_id: int, observations: pd.DataFrame) -> np.ndarray:
    """
    Show the mean-square error of standard 1-held out estimates of the effect at location location_id from observations

    :parm location_id: which location to calculate for
    :param observations: the observations data frame
    :return: 1-held out standard estimates
    """
    # get observations in target group
    obs = observations.loc[observations["location_id"] == location_id, :].reset_index(drop=True, inplace=False)
    # get the standard 1-held out estimates of the effect
    observed_means = np.array([np.mean(obs.loc[obs.index != i, "observations"]) for i in range(obs.shape[0])])
    return observed_means   

In [21]:
# estimate the quality of the standard estimator
def get_1_held_out_sq_error_of_direct_estimate_loss(location_id: int) -> float:
    """
    Show the mean-square error of standard 1-held out estimates of the effect at location location_id from observations

    :parm location_id: which location to calculate for
    :return: estimated mean square error of this estimation procedure
    """
    # get the estimates
    observed_means = standard_effect_estimate_1_held_out(location_id=location_id, observations=observations)
    # get the unobservable true effect for comparison
    true_effect = example_locations.loc[example_locations["location_id"] == location_id, "effect"].values[0]
    # calculate the mean square error of these estimates
    mean_square_error = np.mean((observed_means - true_effect)**2)
    return mean_square_error


In [22]:
# define the pooled estimator
# a lot of the code complexity is the 1-hold out procedures
# which are not needed to build the estimate, but to evaluate the quality of the estimate.
def pooled_effect_estimate_1_held_out(*, location_id: int, observations: pd.DataFrame) -> np.ndarray:
    """
    Show the mean-square error of pooled 1-held out estimates of the effect at location location_id from observations

    :parm location_id: which location to calculate for
    :param observations: the observations data frame
    :return: 1-held out pooled estimates
    """
    # get observations in target group
    obs = observations.loc[observations["location_id"] == location_id, :].reset_index(drop=True, inplace=False)
    # get the standard 1-held out estimates of the effect
    observed_means = np.array([np.mean(obs.loc[obs.index != i, "observations"]) for i in range(obs.shape[0])])
    # get the observed variance between locations
    location_centers = observations.loc[:, ["location_id", "observations"]].groupby(["location_id"]).mean().reset_index(drop=False, inplace=False)
    location_centers.rename(columns={"observations": "centers"}, inplace=True)
    between_location_var = np.var(location_centers["centers"], ddof=1)
    # get the observed variance for each locations, excluding own location to make the out-of same
    combined = observations.merge(
        location_centers,
        on=["location_id"],
        how="left",
        )
    combined = combined.loc[combined["location_id"] != location_id, :].reset_index(drop=True, inplace=False)
    observation_var = np.sum((combined["observations"] - combined["centers"])**2) / (combined.shape[0] - len(set(combined["location_id"])))
    # get the mean of all observations, except self to strengthen hold-out strictness.
    grand_mean = np.mean(combined["observations"])
    # get v, the pooling coefficient
    n_j = obs.shape[0] - 1  # minus 1 to account for hold-out
    v = 1 / (1 + observation_var / ((n_j + 1) * between_location_var))
    # build the held-out pooled estimate
    pooled_estimate = v * observed_means + (1 - v) * grand_mean
    return pooled_estimate

In [23]:
# evaluate the pooled estimator quality
def get_1_held_out_pooled_estimate_loss(location_id: int) -> float:
    """
    Show the mean-square error of partial pooled 1-held out estimates of the effect at location location_id from observations

    :parm location_id: which location to calculate for
    :return: estimated mean square error of this estimation procedure
    """
    # get the estimates
    pooled_estimate = pooled_effect_estimate_1_held_out(location_id=location_id, observations=observations)
    # get the unobservable true effect for comparison
    true_effect = example_locations.loc[example_locations["location_id"] == location_id, "effect"].values[0]
    # calculate the mean square error of these estimates
    mean_square_error = np.mean((pooled_estimate - true_effect)**2)
    return mean_square_error


Now, let's try the estimators. In each case we the evaluation is returning mean-square-error, so smaller is better.

In [24]:
# run the experiment for the standard estimator
std_est_loss = get_1_held_out_sq_error_of_direct_estimate_loss(0)

std_est_loss

42.00943220599963

In [25]:
# run the experiment for the pooled estimator
pooled_est_loss = get_1_held_out_pooled_estimate_loss(0)

pooled_est_loss

20.265085844392072

In [26]:
assert pooled_est_loss < std_est_loss

And we see, the pooled estimate has smaller expected 1-hold out mean square error loss that the standard estimate. That is: we have evidence the pooled estimate would be closer to new observations for a given location.

The method has been seen to work.

We used hold-out procedures on this data so we could estimate the future out of sample performance of the estimate. That is: to get a reliable estimate of if new observations would be near the inferred values or not. In production we would then run the procedure again with no hold-out to get a more statistically efficient estimate that is using more of the available data.

## Conclusion

The partial pooling improvement for estimating an unseen value from noisy observations depends on a single parameter `PerObservationSD**2 / ((n_j + 1) * BetweenLocationsSD**2)`. This parameter compares the uncertainty in the observations from a single location, to the uncertainty per-location, scaled by how many observations we have at the location in question. When this ratio is small, we don't pool data- we just estimate the average value using data from one location. When this ratio is large, pooling is likely a useful variance reducing procedure.

Essentially the method is trading away variance (due to having very few samples) for bias (due to including samples that don't match the target location). If the different locations are sufficiently related, this can be an improving trade off in terms of expected square error.

In practice the above inference is made inside a hierarchical model solver. However, it is good to see the expect form of the pooling strategy.